In [1]:
# Step 1: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:

# Step 2: Load the dataset
data = pd.read_excel('C:/Users/Prachi/Documents/Data Science Masters/Thesis/students_with_total_scores.xlsx')


In [3]:

# Step 3: Define feature columns (use 'answer classes' columns or 'punkte' columns)
feature_columns = [col for col in data.columns if  'punkte' in col.lower()]  # Adjust this list as per your column names
print(feature_columns)


['Punkte', 'Punkte.1', 'Punkte.2', 'Punkte.3', 'Punkte.4', 'Punkte.5', 'Punkte.6', 'Punkte.7', 'Punkte.8', 'Punkte.9', 'Punkte.10', 'Punkte.11', 'Punkte.12', 'Punkte.13', 'Punkte.14', 'Punkte.15', 'Punkte.16', 'Punkte.17', 'Punkte.18', 'Punkte.19', 'Punkte.20', 'Punkte.21', 'Punkte.22', 'Punkte.23', 'Punkte.24', 'Punkte.25', 'Punkte.26', 'Punkte.27', 'Punkte.28', 'Punkte.29', 'Punkte.30', 'Punkte.31', 'Punkte.32', 'Punkte.33', 'Punkte.34', 'Punkte.35', 'Punkte.36', 'Punkte.37', 'Punkte.38', 'Punkte.39', 'Punkte.40', 'Punkte.41', 'Punkte.42']


In [4]:
data['Pass_Fail'] = data['Total_Score'].apply(lambda x: 1 if x > 40 else 0)

In [5]:

# Step 4: Define the target column
target_column = 'Total_Score'


In [6]:

# Step 5: Prepare the data
X_punkte = [col for col in data.columns if 'punkte' in col.lower()]
# Identify the columns with the naming pattern like '1.1.1' or '1.2' in the original data that need to be updated
X_ansclassns_to_update = [col for col in data.columns if any(part.isdigit() for part in col.split('.'))]
# Remove 'punkte' columns from the answer class columns to update
X_ansclassns_to_update = [col for col in X_ansclassns_to_update if col not in X_punkte]
print(len(X_ansclassns_to_update))
X = data[X_ansclassns_to_update]

y = data[target_column]



43


In [7]:
print(len(X))
print(len(y))

448
448


In [8]:

from sklearn.impute import SimpleImputer

# Create an imputer object with a strategy to fill missing values
imputer = SimpleImputer(strategy='mean')  # You can also use 'median' or 'most_frequent'

# Fit the imputer on the training data and transform it
X_imputed = imputer.fit_transform(X)
# Step 6: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.3, random_state=42)





In [9]:
print(X_train)

[[3. 2. 2. ... 4. 4. 4.]
 [3. 1. 3. ... 2. 2. 2.]
 [4. 5. 3. ... 1. 1. 4.]
 ...
 [4. 1. 3. ... 4. 4. 1.]
 [4. 1. 4. ... 4. 1. 1.]
 [2. 2. 2. ... 4. 4. 2.]]


In [10]:
# Step 7: Train the model
model = LinearRegression()
model.fit(X_train, y_train)



LinearRegression()

In [14]:
from sklearn.linear_model import Ridge, Lasso

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)


Lasso(alpha=0.1)

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict using Ridge
y_pred_ridge = ridge_model.predict(X_test)
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_r2 = r2_score(y_test, y_pred_ridge)

print(f'Ridge Model - Mean Squared Error: {ridge_mse}')
print(f'Ridge Model - R-squared: {ridge_r2}')

# Predict using Lasso
y_pred_lasso = lasso_model.predict(X_test)
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_r2 = r2_score(y_test, y_pred_lasso)

print(f'Lasso Model - Mean Squared Error: {lasso_mse}')
print(f'Lasso Model - R-squared: {lasso_r2}')


Ridge Model - Mean Squared Error: 58.34042340136158
Ridge Model - R-squared: 0.7815665235944578
Lasso Model - Mean Squared Error: 57.94143549603525
Lasso Model - R-squared: 0.7830603817141465


In [16]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Random Forest predictions and evaluation
y_pred_rf = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

print(f'Random Forest - Mean Squared Error: {rf_mse}')
print(f'Random Forest - R-squared: {rf_r2}')


Random Forest - Mean Squared Error: 41.600323119659826
Random Forest - R-squared: 0.8442434478730734


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from the grid search
print(f'Best parameters found: {grid_search.best_params_}')

# Use the best model for prediction
best_rf = grid_search.best_estimator_

# Evaluate the tuned Random Forest model
y_pred_rf = best_rf.predict(X_test)
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

print(f'Random Forest (Tuned) - Mean Squared Error: {rf_mse}')
print(f'Random Forest (Tuned) - R-squared: {rf_r2}')


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\Prachi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
308 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Prachi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Prachi\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Prachi\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\Prachi\anaconda3\Lib\site-

Best parameters found: {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Random Forest (Tuned) - Mean Squared Error: 37.96900888796694
Random Forest (Tuned) - R-squared: 0.8578395197783568


In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


# Initialize the individual models
ridge_model = Ridge(alpha=1.0)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Train the individual models
ridge_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Create the ensemble using VotingRegressor
ensemble_model = VotingRegressor(estimators=[
    ('ridge', ridge_model),
    ('rf', rf_model),
    ('xgb', xgb_model)
])

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Predict using the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test)

# Evaluate the ensemble model
ensemble_mse = mean_squared_error(y_test, y_pred_ensemble)
ensemble_r2 = r2_score(y_test, y_pred_ensemble)

print(f'Ensemble Model - Mean Squared Error: {ensemble_mse}')
print(f'Ensemble Model - R-squared: {ensemble_r2}')


In [11]:
# Step 8: Make predictions
y_pred = model.predict(X_test)



In [13]:
# Step 9: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 58.37546641777471
R-squared: 0.7814353183776874
